In [16]:
import time
import logging
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def obtener_lista_encuestas_dropdown():
    """
    Usa un enfoque semiautomático para que el usuario prepare la página.
    Luego, el script hace clic en el dropdown de 'Select2', espera a que
    las opciones aparezcan y las extrae a un DataFrame.
    """
    url = "https://proyectos.inei.gob.pe/microdatos/"
    print(f"Abriendo el navegador en: {url}")

    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 20)

        # --- PAUSA PARA INTERVENCIÓN MANUAL ---
        input(
            "\n" + "="*50 +
            "\nPAUSA: El navegador está bajo tu control." +
            "\n   1. Cierra el pop-up de anuncio." +
            "\n   2. Haz clic en la pestaña 'Consulta por Encuestas'." +
            "\n\nUna vez que veas el dropdown '(Elija una Encuesta)', regresa aquí y presiona 'Enter'..." +
            "\n" + "="*50 + "\n"
        )
        
        logging.info("Reanudando el script. Extrayendo datos del dropdown...")

        # --- EXTRACCIÓN DE OPCIONES DEL DROPDOWN ---
        # 1. Hacer clic en el dropdown para abrirlo
        # Este es el elemento que tú encontraste.
        dropdown_trigger = wait.until(
            EC.element_to_be_clickable((By.ID, "select2-cmbEncuesta0ID-container"))
        )
        dropdown_trigger.click()
        logging.info("Dropdown de encuestas abierto.")
        
        # 2. Esperar a que las opciones aparezcan y obtenerlas
        # Las opciones de Select2 suelen aparecer en una lista <ul> con esta clase
        options_xpath = "//ul[contains(@class, 'select2-results__options')]/li"
        wait.until(EC.presence_of_element_located((By.XPATH, options_xpath)))
        
        # Leemos todos los elementos <li> de la lista de resultados
        option_elements = driver.find_elements(By.XPATH, options_xpath)
        
        nombres_encuestas = []
        for option in option_elements:
            # Extraemos el texto de cada opción
            texto_opcion = option.text
            # Ignoramos las opciones vacías o que son placeholders
            if texto_opcion and texto_opcion != "(Elija una Encuesta)":
                nombres_encuestas.append(texto_opcion)

        logging.info(f"Se extrajeron {len(nombres_encuestas)} nombres de encuestas del dropdown.")
        
        if not nombres_encuestas:
            logging.error("No se encontraron nombres de encuestas en el dropdown.")
            return None
        
        # 3. Crear el DataFrame final
        # Nota: Desde el dropdown solo podemos obtener el nombre. Las otras columnas no están disponibles aquí.
        df = pd.DataFrame(nombres_encuestas, columns=['Nombre_Encuesta'])
        df['Entidad_Ejecutora'] = 'No disponible desde el dropdown'
        df['Anio_Publicacion'] = 'No disponible desde el dropdown'

        logging.info(f"¡Éxito! Se creó el DataFrame con {len(df)} encuestas.")
        return df

    except Exception as e:
        logging.error("Ocurrió un error durante la ejecución del script.", exc_info=True)
        return None
    finally:
        logging.info("Proceso finalizado. Cerrando el navegador.")
        driver.quit()

# --- Ejecución ---
lista_de_encuestas = obtener_lista_encuestas_dropdown()

if lista_de_encuestas is not None:
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_colwidth', None)
    print("\n--- LISTA DE ENCUESTAS DISPONIBLES EN EL DROPDOWN DEL INEI ---")
    print(lista_de_encuestas)

2025-09-30 11:41:46,613 - INFO - ====== WebDriver manager ======


Abriendo el navegador en: https://proyectos.inei.gob.pe/microdatos/


2025-09-30 11:41:51,028 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-30 11:41:51,344 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-30 11:41:51,582 - INFO - Driver [C:\Users\Alvaro\.wdm\drivers\chromedriver\win64\140.0.7339.207\chromedriver-win32/chromedriver.exe] found in cache
2025-09-30 11:42:03,301 - INFO - Reanudando el script. Extrayendo datos del dropdown...
2025-09-30 11:42:03,564 - INFO - Dropdown de encuestas abierto.
2025-09-30 11:42:04,822 - INFO - Se extrajeron 53 nombres de encuestas del dropdown.
2025-09-30 11:42:04,854 - INFO - ¡Éxito! Se creó el DataFrame con 53 encuestas.
2025-09-30 11:42:04,856 - INFO - Proceso finalizado. Cerrando el navegador.



--- LISTA DE ENCUESTAS DISPONIBLES EN EL DROPDOWN DEL INEI ---
                                                                                        Nombre_Encuesta  \
0                                                                            ENAHO Metodología ANTERIOR   
1                                                                         ENAHO Metodología ACTUALIZADA   
2                                                         ENCUESTA PERMANENTE DE EMPLEO NACIONAL - EPEN   
3                                                                    CENSO DE INFRAESTRUCTURA EDUCATIVA   
4                                                                 CENSO NACIONAL AGROPECUARIO - CENAGRO   
5                           CENSO NACIONAL AGROPECUARIO - CENAGRO - HOJAS DE INFORMACIÓN COMPLEMENTARIA   
6                                                                          CENSO NACIONAL DE COMISARIAS   
7                                                        CENSO NACIONAL DE MERCA

In [ ]:
# Lista COMPLETA de encuestas de interés que vamos a intentar descargar.
# El script iterará sobre esta lista, seleccionando cada una en el dropdown.
target_encuestas = [
    # --- Prioridad Alta ---
    "ENAHO Metodología ACTUALIZADA",
    "ENCUESTA DEMOGRÁFICA Y DE SALUD FAMILIAR - ENDES",
    "MAPA DE POBREZA",
    "ENCUESTA NACIONAL DE PROGRAMAS PRESUPUESTALES - ENAPRES",
    
    # --- Prioridad Secundaria ---
    "REGISTRO NACIONAL DE MUNICIPALIDADES - RENAMU",
    "CENSO NACIONAL AGROPECUARIO - CENAGRO",
    "CENSO NACIONAL DE MERCADOS DE ABASTOS - CENAMA",
    "ENCUESTA NACIONAL ESPECIALIZADA SOBRE DISCAPACIDAD - ENEDIS",
    "ENAHO Metodología ANTERIOR" 
]

print(f"Se intentará descargar un total de {len(target_encuestas)} encuestas:")
for i, encuesta in enumerate(target_encuestas):
    print(f"{i+1}. {encuesta}")
    

Se intentará descargar un total de 9 encuestas:
1. ENAHO Metodología ACTUALIZADA
2. ENCUESTA DEMOGRÁFICA Y DE SALUD FAMILIAR - ENDES
3. MAPA DE POBREZA
4. ENCUESTA NACIONAL DE PROGRAMAS PRESUPUESTALES - ENAPRES
5. REGISTRO NACIONAL DE MUNICIPALIDADES - RENAMU
6. CENSO NACIONAL AGROPECUARIO - CENAGRO
7. CENSO NACIONAL DE MERCADOS DE ABASTOS - CENAMA
8. ENCUESTA NACIONAL ESPECIALIZADA SOBRE DISCAPACIDAD - ENEDIS
9. ENAHO Metodología ANTERIOR


In [5]:
import os
import time
import logging
import pandas as pd
import zipfile
import shutil
import re
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ==============================================================================
# --- CONFIGURACIÓN ---
# ==============================================================================
BASE_SAVE_DIR = r'D:\UNIDAD D\UNIVERSIDAD\2025-2\Modelos Fisiológicos\Modelos_fisio\datasets\INEI_Datasets'
FORMATO_PREFERIDO = 'STATA'

# Dejamos la lista completa, pero usaremos un 'break' para probar solo la primera
target_encuestas = [
    "ENAHO Metodología ACTUALIZADA",
    "ENAHO Metodología ANTERIOR",
    "ENCUESTA DEMOGRÁFICA Y DE SALUD FAMILIAR - ENDES",
    # ... (el resto de encuestas)
]

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# --- FUNCIONES DE AYUDA (Probadas y funcionando) ---
def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name).replace(' ', '_').strip()

def get_options_from_select(select_element):
    options = [opt.get_attribute('text') for opt in select_element.find_elements(By.TAG_NAME, 'option')]
    return [opt for opt in options if opt and not opt.startswith('<') and not opt.startswith('(') and opt.strip() != '-']

def wait_for_spinner(wait):
    wait.until(EC.invisibility_of_element_located((By.ID, "div_espera")))
    time.sleep(1)

def process_single_download(temp_dir, final_path_info):
    try:
        start_time, timeout, zip_path = time.time(), 600, None
        while time.time() - start_time < timeout:
            zips = list(Path(temp_dir).glob('*.zip'))
            if zips and not list(Path(temp_dir).glob('*.crdownload')):
                time.sleep(5); zip_path = zips[0]; break
            time.sleep(1)
        if not zip_path: logging.warning("     -> Timeout en la descarga."); return
        extract_folder = Path(final_path_info['dir']) / zip_path.stem
        final_csv_path = Path(final_path_info['dir']) / (final_path_info['name'] + ".csv")
        os.makedirs(final_path_info['dir'], exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zf: zf.extractall(extract_folder)
        data_file = next(extract_folder.glob('**/*.dta'), None) or next(extract_folder.glob('**/*.sav'), None)
        if data_file:
            df = pd.read_stata(data_file, convert_categoricals=False) if data_file.suffix == '.dta' else pd.read_spss(data_file)
            df.to_csv(final_csv_path, index=False)
            logging.info(f"     -> ÉXITO: Guardado como {final_csv_path.name}")
        else: logging.warning("     -> No se encontró archivo .dta o .sav en el zip.")
    except Exception as e: logging.error(f"     -> Fallo en proceso de archivo: {e}")
    finally:
        if 'zip_path' in locals() and zip_path and zip_path.exists(): zip_path.unlink()
        if 'extract_folder' in locals() and extract_folder and extract_folder.exists(): shutil.rmtree(extract_folder)

# --- SCRIPT PRINCIPAL ---
def main_downloader_final():
    TEMP_DOWNLOAD_DIR = Path(BASE_SAVE_DIR) / 'temp_downloads'
    os.makedirs(TEMP_DOWNLOAD_DIR, exist_ok=True)
    opts = webdriver.ChromeOptions()
    prefs = {"download.default_directory": str(TEMP_DOWNLOAD_DIR)}; opts.add_experimental_option("prefs", prefs)
    opts.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    try:
        driver.get("https://proyectos.inei.gob.pe/microdatos/")
        wait = WebDriverWait(driver, 20) # Reducimos el wait para ver fallos más rápido
        input("\n" + "="*50 + "\nPAUSA: Prepara el navegador.\n   1. Cierra el pop-up.\n   2. Clic en 'Consulta por Encuestas'.\n\nPresiona 'Enter' para continuar...\n" + "="*50 + "\n")
        
        is_first_run = True
        for survey_name in target_encuestas:
            logging.info(f"\n{'='*20} PROCESANDO: {survey_name} {'='*20}")
            try:
                if not is_first_run:
                    logging.info("Reseteando formulario para la siguiente encuesta...")
                    wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Presentación"))).click(); wait_for_spinner(wait)
                    wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Consulta por Encuestas"))).click(); wait_for_spinner(wait)
                
                logging.info("Paso 1: Abriendo dropdown de encuestas...")
                wait.until(EC.element_to_be_clickable((By.ID, "select2-cmbEncuesta0ID-container"))).click()
                
                logging.info(f"Paso 2: Buscando la opción '{survey_name}'...")
                option_xpath = f"//ul[contains(@class, 'select2-results__options')]//li[text()='{survey_name}']"
                option_element = wait.until(EC.element_to_be_clickable((By.XPATH, option_xpath)))
                
                logging.info("Paso 3: Haciendo clic en la opción...")
                option_element.click(); wait_for_spinner(wait)
                logging.info("-> Selección de encuesta principal exitosa.")
                is_first_run = False # A partir de ahora, siempre se reseteará

            except Exception as e:
                logging.error(f"Fallo crítico al seleccionar '{survey_name}'. Saltando. Error: {e}"); continue

            # ... Lógica de descarga por módulos (probada y funciona)
            if "ENAHO" in survey_name:
                sub_survey_xpath = f"//div[@id='{'ENAHON' if 'ACTUALIZADA' in survey_name else 'ENAHOA'}']//select"
                wait.until(EC.presence_of_element_located((By.XPATH, f"{sub_survey_xpath}/option[2]")))
                sub_surveys = get_options_from_select(driver.find_element(By.XPATH, sub_survey_xpath))
                print(f"  Sub-Encuestas encontradas: {sub_surveys}")
                for sub_survey in sub_surveys:
                    logging.info(f"  Procesando sub-encuesta: {sub_survey}")
                    Select(driver.find_element(By.XPATH, sub_survey_xpath)).select_by_visible_text(sub_survey); wait_for_spinner(wait)
                    year_xpath = "//select[@name='cmbAnno']"
                    wait.until(EC.presence_of_element_located((By.XPATH, f"{year_xpath}/option[2]")))
                    years = get_options_from_select(driver.find_element(By.XPATH, year_xpath))
                    print(f"    Años encontrados para '{sub_survey}': {years}")
                    for year in years:
                        Select(driver.find_element(By.XPATH, year_xpath)).select_by_visible_text(year); wait_for_spinner(wait)
                        period_xpath = "//div[@id='divPeriodo']/select"
                        wait.until(EC.presence_of_element_located((By.XPATH, f"{period_xpath}/option[2]")))
                        periods = get_options_from_select(driver.find_element(By.XPATH, period_xpath))
                        print(f"      Períodos encontrados para '{year}': {periods}")
                        for period in periods:
                            logging.info(f"      Procesando período: {period}")
                            Select(driver.find_element(By.XPATH, period_xpath)).select_by_visible_text(period); wait_for_spinner(wait)
                            wait.until(EC.visibility_of_element_located((By.ID, "divDetalle")))
                            module_rows = driver.find_elements(By.XPATH, "//div[@id='divDetalle']//table/tbody/tr/td/table/tbody/tr")
                            for row in module_rows[1:]:
                                try:
                                    cells = row.find_elements(By.TAG_NAME, 'td');
                                    if len(cells) < 7: continue
                                    module_name = cells[6].text.strip()
                                    logging.info(f"        -> Descargando Módulo: {module_name}")
                                    download_link = row.find_element(By.XPATH, f".//a[@title='Archivo {FORMATO_PREFERIDO}']")
                                    for f in Path(TEMP_DOWNLOAD_DIR).glob('*'): f.unlink()
                                    driver.execute_script("arguments[0].click();", download_link)
                                    path_info = {'dir': Path(BASE_SAVE_DIR)/sanitize_filename(survey_name)/sanitize_filename(sub_survey)/year/sanitize_filename(period), 'name': f"MOD_{sanitize_filename(module_name)}"}
                                    process_single_download(TEMP_DOWNLOAD_DIR, path_info)
                                except Exception: continue

            # --- SOLO PARA PRUEBAS: Detener después de la primera encuesta ---
            logging.info(f"Prueba para '{survey_name}' completada. Deteniendo el script.")
            break

    except Exception as e: logging.error("Ocurrió un error fatal.", exc_info=True)
    finally:
        input("Proceso de prueba terminado. Presiona Enter para cerrar el navegador.")
        driver.quit()
        if TEMP_DOWNLOAD_DIR.exists(): shutil.rmtree(TEMP_DOWNLOAD_DIR)
        logging.info("--- PROCESO FINALIZADO ---")

if __name__ == "__main__":
    main_downloader_final()

2025-09-30 21:24:25,530 - INFO - ====== WebDriver manager ======
2025-09-30 21:24:26,390 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-30 21:24:26,533 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-30 21:24:26,680 - INFO - Driver [C:\Users\Alvaro\.wdm\drivers\chromedriver\win64\140.0.7339.207\chromedriver-win32/chromedriver.exe] found in cache
2025-09-30 21:24:32,071 - INFO - 
==================== PROCESANDO: ENAHO Metodología ACTUALIZADA ====================
2025-09-30 21:24:32,073 - INFO - Paso 1: Abriendo dropdown de encuestas...
2025-09-30 21:24:32,177 - INFO - Paso 2: Buscando la opción 'ENAHO Metodología ACTUALIZADA'...
2025-09-30 21:24:32,208 - INFO - Paso 3: Haciendo clic en la opción...
2025-09-30 21:24:33,284 - INFO - -> Selección de encuesta principal exitosa.
2025-09-30 21:24:33,343 - INFO -   Procesando sub-encuesta: Condiciones de Vida y Pobreza - ENAHO


  Sub-Encuestas encontradas: ['Condiciones de Vida y Pobreza - ENAHO', 'Condiciones de vida y pobreza - ENAHO PANEL']
    Años encontrados para 'Condiciones de Vida y Pobreza - ENAHO': ['2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']


2025-09-30 21:24:35,693 - INFO -       Procesando período: Anual - (Ene-Dic)


      Períodos encontrados para '2004': ['Anual - (Ene-Dic)']


2025-09-30 21:24:36,820 - INFO -         -> Descargando Módulo: Características de la Vivienda y del Hogar
2025-09-30 21:24:46,919 - INFO -      -> ÉXITO: Guardado como MOD_Características_de_la_Vivienda_y_del_Hogar.csv
2025-09-30 21:24:46,948 - INFO -         -> Descargando Módulo: Características de los Miembros del Hogar
2025-09-30 21:25:09,992 - INFO -      -> ÉXITO: Guardado como MOD_Características_de_los_Miembros_del_Hogar.csv
2025-09-30 21:25:10,031 - INFO -         -> Descargando Módulo: Educación
2025-09-30 21:25:24,811 - INFO -      -> ÉXITO: Guardado como MOD_Educación.csv
2025-09-30 21:25:24,862 - INFO -         -> Descargando Módulo: Salud
2025-09-30 21:25:33,135 - ERROR -      -> Fallo en proceso de archivo: Version of given Stata file is 110. pandas supports importing versions 105, 108, 111 (Stata 7SE), 113 (Stata 8/9), 114 (Stata 10/11), 115 (Stata 12), 117 (Stata 13), 118 (Stata 14/15/16),and 119 (Stata 15/16, over 32,767 variables).
2025-09-30 21:25:33,161 - INFO -  

...

2025-09-30 21:56:15,815 - INFO -         -> Descargando Módulo: Características de la Vivienda y del Hogar
2025-09-30 21:56:27,638 - INFO -      -> ÉXITO: Guardado como MOD_Características_de_la_Vivienda_y_del_Hogar.csv
2025-09-30 21:56:27,681 - INFO -         -> Descargando Módulo: Características de los Miembros del Hogar
2025-09-30 21:56:36,140 - INFO -      -> ÉXITO: Guardado como MOD_Características_de_los_Miembros_del_Hogar.csv
2025-09-30 21:56:36,178 - INFO -         -> Descargando Módulo: Educación
2025-09-30 21:56:51,915 - INFO -      -> ÉXITO: Guardado como MOD_Educación.csv
2025-09-30 21:56:51,970 - INFO -         -> Descargando Módulo: Salud
2025-09-30 21:57:19,690 - INFO -      -> ÉXITO: Guardado como MOD_Salud.csv
2025-09-30 21:57:19,772 - INFO -         -> Descargando Módulo: Empleo e Ingresos
2025-09-30 21:57:51,802 - INFO -      -> ÉXITO: Guardado como MOD_Empleo_e_Ingresos.csv
2025-09-30 21:57:51,933 - INFO -         -> Descargando Módulo: Gastos en Alimentos y Bebida

      Períodos encontrados para '2012': ['Anual - (Ene-Dic)', 'Trimestre 1', 'Trimestre 2', 'Trimestre 3', 'Trimestre 4']


2025-09-30 22:06:22,668 - INFO -         -> Descargando Módulo: Características de la Vivienda y del Hogar
2025-09-30 22:06:33,368 - INFO -      -> ÉXITO: Guardado como MOD_Características_de_la_Vivienda_y_del_Hogar.csv
2025-09-30 22:06:33,415 - INFO -         -> Descargando Módulo: Características de los Miembros del Hogar
2025-09-30 22:06:41,673 - INFO -      -> ÉXITO: Guardado como MOD_Características_de_los_Miembros_del_Hogar.csv
2025-09-30 22:06:41,713 - INFO -         -> Descargando Módulo: Educación
2025-09-30 22:07:01,390 - INFO -      -> ÉXITO: Guardado como MOD_Educación.csv
2025-09-30 22:07:01,531 - INFO -         -> Descargando Módulo: Salud
2025-09-30 22:07:07,971 - ERROR -      -> Fallo en proceso de archivo: Version of given Stata file is 110. pandas supports importing versions 105, 108, 111 (Stata 7SE), 113 (Stata 8/9), 114 (Stata 10/11), 115 (Stata 12), 117 (Stata 13), 118 (Stata 14/15/16),and 119 (Stata 15/16, over 32,767 variables).
2025-09-30 22:07:08,005 - INFO -  

      Períodos encontrados para '2013': ['Anual - (Ene-Dic)', 'Trimestre 1', 'Trimestre 2', 'Trimestre 3', 'Trimestre 4']


2025-09-30 22:14:31,230 - INFO -         -> Descargando Módulo: Características de la Vivienda y del Hogar
2025-09-30 22:14:42,725 - INFO -      -> ÉXITO: Guardado como MOD_Características_de_la_Vivienda_y_del_Hogar.csv
2025-09-30 22:14:42,770 - INFO -         -> Descargando Módulo: Características de los Miembros del Hogar
2025-09-30 22:14:51,188 - INFO -      -> ÉXITO: Guardado como MOD_Características_de_los_Miembros_del_Hogar.csv
2025-09-30 22:14:51,232 - INFO -         -> Descargando Módulo: Educación
2025-09-30 22:15:09,128 - INFO -      -> ÉXITO: Guardado como MOD_Educación.csv
2025-09-30 22:15:09,196 - INFO -         -> Descargando Módulo: Salud
2025-09-30 22:15:38,693 - INFO -      -> ÉXITO: Guardado como MOD_Salud.csv
2025-09-30 22:15:38,786 - INFO -         -> Descargando Módulo: Empleo e Ingresos
2025-09-30 22:16:16,936 - INFO -      -> ÉXITO: Guardado como MOD_Empleo_e_Ingresos.csv
2025-09-30 22:16:17,049 - INFO -         -> Descargando Módulo: Gastos en Alimentos y Bebida